In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import selectivesearch
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [2]:
cars = r"C:\Users\hp\Documents\maskRCNN\car"
bus = r"C:\Users\hp\Documents\maskRCNN\bus"
ambulance = r"C:\Users\hp\Documents\maskRCNN\ambulance"
motorcycle = r"C:\Users\hp\Documents\maskRCNN\motorcycle"
truck = r"C:\Users\hp\Documents\maskRCNN\truck"

In [3]:
car_images = []
car_labels = []
for image_file in os.listdir(cars):
    images1 = cv2.imread(os.path.join(cars, image_file))
    # images1 = cv2.resize(images1, (64, 64))  # Resize the image to 64x64 pixels
    # images1 = images1.astype('float32') / 255.0
    car_images.append(images1)
    car_labels.append(1)  # Assign label 1 for car class



In [5]:
bus_images = []
bus_labels = []
for image_file in os.listdir(bus):
    images2 = cv2.imread(os.path.join(bus, image_file))
    # images2 = cv2.resize(images2, (64, 64))  # Resize the image to 64x64 pixels
    # images2 = images2.astype('float32') / 255.0
    bus_images.append(images2)
    bus_labels.append(2)  # Assign label 2 for bus class

In [6]:
amb_images = []
amb_labels = []
for image_file in os.listdir(ambulance):
    images3 = cv2.imread(os.path.join(ambulance, image_file))
    # images3 = cv2.resize(images3, (64, 64))  # Resize the image to 64x64 pixels
    # images3 = images3.astype('float32') / 255.0
    amb_images.append(images3)
    amb_labels.append(3)  # Assign label 3 for vehicle class

In [7]:
motor_images = []
motor_labels = []
for image_file in os.listdir(motorcycle):
    images4 = cv2.imread(os.path.join(motorcycle, image_file))
    # images4 = cv2.resize(images1, (64, 64))  # Resize the image to 64x64 pixels
    # images4 = images4.astype('float32') / 255.0
    motor_images.append(images4)
    motor_labels.append(4)  # Assign label 1 for vehicle class

In [9]:
truck_images = []
truck_labels = []
for image_file in os.listdir(truck):
    images5 = cv2.imread(os.path.join(truck, image_file))
    # images5 = cv2.resize(images1, (64, 64))  # Resize the image to 64x64 pixels
    # images5 = images5.astype('float32') / 255.0
    truck_images.append(images5)
    truck_labels.append(5)  # Assign label 1 for vehicle class

In [10]:
#  Concatenate images and labels
all_images = car_images + bus_images + truck_images +amb_images + motor_images
all_labels = car_labels + bus_labels + truck_labels + amb_labels + motor_labels

# Split the dataset into train and test sets
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.2, stratify=all_labels, random_state=42)

In [11]:
print(images4.shape)

(416, 416, 3)


In [12]:
# Perform region proposal generation using Selective Search
def generate_region_proposals(images):
    proposals = []
    for image in images:
        # Convert image to RGB format
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Apply Selective Search algorithm to generate region proposals
        # Perform Selective Search
        img_lbl, regions = selectivesearch.selective_search(image_rgb, scale=500, sigma=0.9, min_size=10)
        
        # Filter out small regions
        filtered_regions = []
        for r in regions:
            x, y, w, h = r['rect']
            if w < 10 or h < 10 or (float(w) / h) > 5 or (float(h) / w) > 5:
                continue
            filtered_regions.append(r)
        
        proposals.append(filtered_regions)
    return proposals

In [13]:
train_proposals = generate_region_proposals(train_images)
test_proposals = generate_region_proposals(test_images)

d:\anaconda\envs\mask-rcnn\lib\site-packages\skimage\feature\texture.py:353: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


In [14]:
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(416, 416, 3))

In [16]:
def extract_features(images, proposals):
    features = []
    for image, regions in zip(images, proposals):
        # Iterate over the region proposals
        for r in regions:
            x, y, w, h = r['rect']
            # Extract region of interest (ROI)
            roi = image[y:y+h, x:x+w]
            # Resize the ROI to match the input size of VGG16
            roi = cv2.resize(roi, (416, 416))
            # Preprocess the ROI according to VGG16 requirements
            preprocessed_roi = preprocess_input(roi)
            # Extract features using VGG16
            feature = vgg_model.predict(np.expand_dims(preprocessed_roi, axis=0)).flatten()
            features.append(feature)
    return features

In [17]:
train_features = extract_features(train_images, train_proposals)
test_features = extract_features(test_images, test_proposals)

1/1 [==============================] - 1s 1s/step



KeyboardInterrupt



In [ ]:
train_features = np.array(train_features)
train_labels = np.array(train_labels)
test_features = np.array(test_features)
test_labels = np.array(test_labels)

In [ ]:
# Train a classifier (SVM) on the extracted features
classifier = SVC()
classifier.fit(train_features, train_labels)

In [ ]:
# Make predictions on the test set
predictions = classifier.predict(test_features)

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy:", accuracy)